In [17]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import locale
locale.setlocale(locale.LC_ALL, "ru") 
import json

### Задание 1 - Обязательная часть

In [2]:
keywords = ['Фриланс', 'парсинг']

In [3]:
pages = ['https://habr.com/ru/all/page' + str(x) for x in range(1,51)]

In [4]:
forcast = pd.DataFrame()

In [5]:
for page in pages:
    # получаем страницу
    req = requests.get(page)
    soup = BeautifulSoup(req.text)
    
    # извлекаем посты
    posts = soup.find_all('article', class_='post post_preview')
    
    # находим посты с нашими ключевыми словами
    for post in posts:
        for key in keywords:
            if key in post.text:
                # извлекаем дату, название статьи, ссылку для записи в Dataframe
                date = post.find('span', class_="post__time").text
                if "сегодня" in date:
                    date = date.replace("сегодня", str(datetime.date.today()))
                elif "вчера" in date:
                    date = date.replace("вчера", str(datetime.date.today() - datetime.timedelta(days=1)))
                link = post.find('a', class_="post__title_link").get('href')
                title = post.find('a', class_="post__title_link").text
                row = {'date':date, 'title': title, 'link': link}
                forcast = pd.concat([forcast, pd.DataFrame([row])])

In [6]:
forcast.head()

,date,title,link
0,24 марта 2021 в 07:33,Заказчик не платит: как защититься или как заб...,https://habr.com/ru/company/itsoft/blog/548474/
0,21 марта 2021 в 13:44,15 заповедей IT-фриланса и мелкой разработки,https://habr.com/ru/post/548142/
0,16 марта 2021 в 12:35,Анонимный запрос,https://habr.com/ru/post/547300/
0,10 марта 2021 в 15:17,О самозанятых и трудящихся. Все о переезде в П...,https://habr.com/ru/post/546196/
0,10 марта 2021 в 15:01,Парсинг — это законно?,https://habr.com/ru/post/545818/


### Задание 2 - Обязательная часть

In [8]:
emails_2 = {'emailAddresses':['xxx@x.ru', 'yyy@y.com']}

In [9]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [10]:
headers =  {'Vaar-Version': '0', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0'}

In [11]:
avast_response = requests.post(url, json = emails_2, headers = headers)


In [22]:
response = json.loads(avast_response.text)

In [85]:
# Сделаем словарь, где ключ - емейл, а значение: список id утечек
response['summary']
emails_and_breaches = {key:list(value.values())[0] for key,value in response['summary'].items()}

In [34]:
emails_and_breaches

{'xxx@x.ru': [3176, 12, 3, 3164, 2961, 15],
 'yyy@y.com': [17110,
  17670,
  13094,
  37177,
  16768,
  41,
  2,
  3587,
  17009,
  3,
  13662,
  3520,
  15,
  3669]}

In [36]:
forcast_2 = pd.DataFrame()

In [86]:
# Итерируемся по всем ошибкам, если находим совпадение с нашим списком (emails_and_breaches), извлекаем данные и формируем dataFrame
for mail, ids in emails_and_breaches.items():
    email_df = mail
    for breach,data in response['breaches'].items():
        if int(breach) in ids:
            date_df = data['publishDate']
            source_df = data['site']
            description_df = data['description']
            row_2 = {'email':email_df, 'date': date_df, 'source': source_df, 'description': description_df}
            forcast_2 = pd.concat([forcast_2, pd.DataFrame([row_2])])  

In [84]:
forcast_2

,email,date,source,description
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2017-11-04T00:00:00Z,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
